# Developer tutorial: Analyzers

<div class="alert alert-info">
    
An interactive version of this notebook is available on [Google Colab](https://colab.research.google.com/github/starsimhub/starsim/blob/main/docs/tutorials/dev_tut_analyzers.ipynb?install=starsim) or [Binder](https://mybinder.org/v2/gh/starsimhub/starsim/HEAD?labpath=docs%2Ftutorials%2Fdev_tut_analyzers.ipynb).
    
</div>

## Reporting results

Each Starsim module can have its own results, which get added to the full list of results in the Sim object. For example, the `ss.Pregnancy` module adds results like `sim.results.pregnancy.pregnant`, and the `ss.HIV` module adds results like `sim.results.hiv.new_infections`. If you are writing your own module, you can add whatever custom results you want. However, another option is to create an `Analyzer` to store results that you might need for one particular analysis but won't need all the time. An `Analyzer` is very similar to other Starsim modules in its structure, but the general idea of an analyzer is that it gets called at the end of a timestep, and reports of the state of things after everything else has been updated without changing any of the module states itself.


## Simple usage

For simple reporting, it's possible to use a single function as an analyzer. In this case, the function receives a single argument, `sim`, which it has full access to. For example, if you wanted to know the number of connections in the network on each timestep, you could write a small analyzer as follows:

In [ ]:
import starsim as ss
import matplotlib.pyplot as plt

# Store the number of edges
n_edges = []

def count_edges(sim):
    """ Print out the number of edges in the network on each timestep """
    network = sim.networks[0] # Get the first network
    n = len(network)
    n_edges.append(n)
    print(f'Number of edges for network {network.name} on step {sim.ti}: {n}')
    return

# Create the sim
pars = dict(
    diseases='sis',
    networks = 'mf',
    analyzers = count_edges,
    demographics = True,
)

# Run the sim
sim = ss.Sim(pars).run()
sim.plot()

# Plot the number of edges
plt.figure()
plt.plot(sim.timevec, n_edges)
plt.title('Number of edges over time')
plt.show()

Is that what you expected it to look like? The reason it looks like that is that initially, agents die (either from aging or from disease), reducing the number of edges. New agents are being born, but they don't participate in male-female networks until the age of debut -- which is 15 years old by default, which is why the trend reverses (and tracks population size) after 2015. This illustrates the importance of model burn-in!

## Advanced usage

Suppose we wanted to create an analyzer that would report on the number of new HIV infections in pregnant women:


In [ ]:
import starsim as ss
import pandas as pd

class HIV_preg(ss.Analyzer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        return
    
    def init_results(self):
        super().init_results()
        self.define_results(
            ss.Result('new_infections_pregnancy'),
        )
        return

    def step(self):
        sim = self.sim
        ti = sim.ti
        hiv = sim.diseases.hiv
        pregnant = sim.demographics.pregnancy.pregnant
        newly_infected = hiv.ti_infected == ti
        self.results['new_infections_pregnancy'][ti] = len((newly_infected & pregnant).uids)
        return

pregnancy = ss.Pregnancy(pars=dict(fertility_rate=pd.read_csv('test_data/nigeria_asfr.csv')))
hiv = ss.HIV(beta={'mfnet':[0.5,0.25]})
sim = ss.Sim(diseases=hiv, networks='mfnet', demographics=pregnancy, analyzers=HIV_preg())
sim.run()
print(f'Total infections among pregnant women: {sim.results.hiv_preg.new_infections_pregnancy.sum()}')


Analyzers are ideal for adding custom results, and because they get added to the sim in the same way as any other result, they also get automatically exported in the same format, e.g. using `sim.to_df()`.

Here's a plot of the results from our HIV in pregnancy analyzer:

In [ ]:
import matplotlib.pyplot as plt

res = sim.results.hiv_preg

plt.figure()
plt.bar(res.timevec, res.new_infections_pregnancy)
plt.title('HIV infections acquired during pregnancy')
plt.show()